# 检索增强知识库智能体 (Enhance Agent RAG)
---


关于 RAG（检索增强生成）优化，我们需要从系统的各个环节入手：数据准备、索引、检索、重排序到最后的生成。

以下是一份结构清晰、实战导向的 RAG 优化指南，适合从入门到进阶的开发者阅读。

> 本教程将按照 RAG 的流水线（Pipeline），分阶段介绍优化策略。

# 阶段一：数据准备与索引 (Indexing Strategy)
核心原则：垃圾进，垃圾出（Garbage In, Garbage Out）。检索质量的上限取决于数据结构。

## 1.优化切片策略 (Advanced Chunking) 

不要盲目使用固定字符数（如 512 token）切分。
语义切片 (Semantic Chunking)： 基于段落结构、Markdown 标题或语义相似度断句，确保一个 Chunk 包含一个完整的信息点。

父子索引 (Parent-Child / Small-to-Big Indexing)： 这是提升效果最显著的技巧之一。
原理： 检索时使用“小切片”（Small Chunk）以获得高匹配度，但给 LLM 上下文时返回该切片所属的“大父文档”（Parent Document）。

优势： 既保证了检索的精准度，又保留了上下文的完整性。
滑动窗口 (Sliding Window)： 让切片之间有重叠（Overlap），防止关键信息被切断。

## 2.元数据增强 (Metadata Injection)
纯文本检索往往不够用，需要给 Chunk 加上标签。

增加过滤字段： 为数据添加 年份、作者、文件类型、部门 等元数据。在检索前先进行 metadata filtering，可以极大缩小搜索范围，提高准确率。

摘要嵌入： 为每个文档生成一个简短摘要并进行 Embedding，有时检索摘要比检索全文切片更准确。

> 这两个检索优化方式，参考之前的RAG问答例子，按空格进行分割 + 常用筛选字段写入Metadata元数据。

# 阶段二：查询预处理 (Query Pre-processing)
核心原则：用户的提问通常是模糊、缺失上下文的，不要直接拿用户的原始 Query 去搜索。

## 1. 查询重写 (Query Rewriting)
使用 LLM 将用户的 Query 改写为更适合检索的形式。

> 例子： 用户问“它多少钱？”，系统结合对话历史改写为“iPhone 15 Pro Max 256GB 的价格是多少？”<br />

> 例子： SQL问数：“请统计每个用户的订单数量”，传统方式是改写为“SELECT user_id, COUNT(order_id) AS order_count FROM orders GROUP BY user_id”。<br />

> 更优雅的方式是机构化为参数，如：fields = ["user_id", "COUNT(order_id) AS order_count"], table = "orders", group_by = ["user_id"]。<br />
> 然后把结构化参数传入MCP查询工具进行查询。

![rag.jpg](./img/rag.jpg)

## 2. 查询扩展与分解 (Query Expansion & Decomposition)
多角度查询 (Multi-Query)： 让 LLM 基于原始问题生成 3-5 个不同角度的相似问题，并行检索，取并集。
这能解决措辞不同导致检索失败的问题。

子问题分解 (Sub-question)： 针对复杂问题（如“对比 A 和 B 的优缺点”），将其拆解为
- “A 的优点”
- “B 的优点”
- “对比分析”<br />
三个子查询分别检索。

## 3. HyDE (Hypothetical Document Embeddings)
原理： 让 LLM 先生成一个“假设性的答案”（哪怕是瞎编的），然后用这个假设答案去向量库检索。<br />
逻辑： 假设答案与真实文档在向量空间中的相似度，往往比“问题 vs 文档”的相似度更高。




阶段三：检索阶段 (Retrieval)
核心原则：不要迷信纯向量检索，混合检索才是王道。
1. 混合检索 (Hybrid Search)
这是生产环境的标配。
向量检索 (Dense Retrieval)： 擅长语义匹配（理解“苹果”和“水果”的关系）。
关键词检索 (Sparse Retrieval / BM25)： 擅长精确匹配（如专有名词、型号代码 Error-503）。
策略： 同时进行两路检索，使用 RRF (Reciprocal Rank Fusion) 算法合并结果。
2. 选择更好的 Embedding 模型
不要默认使用 OpenAI 的 text-embedding-3。
查看 MTEB Leaderboard（HuggingFace 上的榜单）。
推荐模型：BGE-M3 (支持多语言、长文本), E5-Mistral, Cohere-embed。
微调 (Fine-tuning)： 如果是垂直领域（如医疗、法律），微调 Embedding 模型能显著提升对特定术语的理解能力。
阶段四：检索后处理 (Post-Retrieval / Reranking)
核心原则：检索是为了召回（Recall），重排序是为了精度（Precision）。
1. 重排序 (Reranking) —— ⭐️ 最重要的优化点
向量检索为了速度牺牲了精度。检索出 Top-50 个结果后，必须使用 Cross-Encoder 模型进行精细打分。
工具： BGE-Reranker, Cohere Rerank, Jina Reranker。
流程： 检索 Top 50 -> Rerank 模型打分 -> 截取 Top 5 给 LLM。
效果： 往往能带来 10%-20% 的准确率提升。
2. 上下文压缩与过滤
去除低相关性的 Chunk。
使用工具（如 LongLLMLingua）压缩 Prompt，移除对回答问题无用的废话，节省 Token 成本并减少对 LLM 的干扰（Lost in the Middle 现象）。
阶段五：生成阶段 (Generation)
核心原则：不仅要回答，还要让用户信任。
1. Prompt 工程优化
限制范围： 明确指令 仅根据提供的上下文回答，如果上下文中没有答案，请说不知道，不要编造。
思维链 (Chain of Thought)： 引导模型先分析上下文，再得出结论。
2. 引用来源 (Citations)
要求 LLM 在回答的每一句话后面标注 [来源: 文档A]。这不仅能减少幻觉，还能让用户方便验证。
阶段六：评估与迭代 (Evaluation)
核心原则：无法量化就无法优化。
不要凭感觉判断 RAG 好不好，要建立自动化测试集。
1. RAGAS (RAG Assessment) 框架
使用 RAGAS 等工具计算核心指标：
Faithfulness (忠实度)： 回答是否忠实于检索到的上下文？（检测幻觉）
Answer Relevance (回答相关性)： 回答是否解决了用户的问题？
Context Precision (上下文精准度)： 检索到的内容里有多少是有用的？
2. 构建“黄金数据集” (Golden Dataset)
准备 50-100 个真实业务场景下的“问题-标准答案”对，每次优化代码后跑一遍测试，观察分数的涨跌。
总结：RAG 优化清单 (Checklist)
如果你遇到效果瓶颈，请按以下顺序排查：
检索内容不对？ -> 检查切片策略（尝试父子索引），上混合检索（BM25+Vector）。
检索排序乱？ -> 加上 Rerank 模型（必做）。
大模型胡说八道？ -> 优化 System Prompt，强约束“不知则不知”。
找不到专有名词？ -> 增加关键词检索权重，或微调 Embedding。
问题太复杂？ -> 引入 Query Agent 做多步推理或问题拆解。
推荐技术栈
框架： LangChain, LlamaIndex (后者在数据索引优化上更强)
向量数据库： Milvus, Pinecone, Weaviate, Qdrant
Rerank 模型： BGE-Reranker-v2-m3, Cohere
评估： RAGAS, TruLens